In [1]:
import numpy as np
import uproot3
import matplotlib.pyplot as plt

In [2]:
%matplotlib widget

In [3]:
infile_21l = 'data/largedrum_21L_dense_MedianCut_3cmVoxel_all.discriminator.root'

In [4]:
fu = uproot3.open(infile_21l)
histogram_21l = fu['histMedianMetric'].numpy()[0]
bins_x, bins_y, bins_z = fu['histMedianMetric;1'].numpy()[1][0]
shape_x, shape_y, shape_z = histogram_21l.shape

#### Calculate bin centre

In [5]:
bincentre_x = []
for i, _ in enumerate(bins_x[:-1]):
    #print(i, bins_x[i], bins_x[i+1],(bins_x[i]+bins_x[i+1])/2 )
    bincentre_x.append((bins_x[i]+bins_x[i+1])/2)
bincentre_x = np.asarray(bincentre_x)

In [6]:
bincentre_y = []
for i, _ in enumerate(bins_y[:-1]):
    bincentre_y.append((bins_y[i]+bins_y[i+1])/2)
bincentre_y = np.asarray(bincentre_y)

In [7]:
bincentre_z = []
for i, _ in enumerate(bins_z[:-1]):
    bincentre_z.append((bins_z[i]+bins_z[i+1])/2)
bincentre_z = np.asarray(bincentre_z)

In [8]:
def select_histogram_coords(histogram, x_min, x_max, y_min, y_max, z_min, z_max):
    
    sel_x = np.where(np.logical_and(bincentre_x>=x_min, bincentre_x<=x_max))[0]#[:-1]
    sel_y = np.where(np.logical_and(bincentre_y>=y_min, bincentre_y<=y_max))[0]#[:-1]
    sel_z = np.where(np.logical_and(bincentre_z>=z_min, bincentre_z<=z_max))[0]#[:-1]

    bincentre_sel_x = bincentre_x[sel_x]
    bincentre_sel_y = bincentre_y[sel_y] 
    bincentre_sel_z = bincentre_z[sel_z]
    #print(sel_x, bincentre_x[sel_x],bincentre_sel_x)
    xx, yy, zz = np.meshgrid(sel_x, sel_y, sel_z, indexing='ij')
    selected_histogram = np.copy(histogram[xx, yy, zz])
    
    return selected_histogram, (bincentre_sel_x, bincentre_sel_y, bincentre_sel_z)

#### select histogram portion (bitumen/bubble)

In [9]:
bitumen_left_hist, bins_left_bitumen = select_histogram_coords(histogram_21l, -440, -200, -300, 300, -300, 300)

In [10]:
bitumen_right_hist, bins_right_bitumen = select_histogram_coords(histogram_21l, 200, 440, -300, 300, -300, 300)

In [11]:
bubble_hist, bins_bubble = select_histogram_coords(histogram_21l, -100, 100, -100, 100, -100, 100)

In [12]:
drum_hist, bins_drum = select_histogram_coords(histogram_21l, -440, 440, -300, 300, -300, 300)

#### for each voxel, calculate distance from the x axis and get metric value

distance from x axis: sqrt(y^2+z^2)

In [13]:
def distance_and_metric(histogram, indices, bins):
    distance_yz = []
    distance_xyz = []
    distance_x = []
    median = []
    indices_x, indices_y, indices_z = indices
    bins_x, bins_y, bins_z = bins
    for x, y, z in zip(indices_x, indices_y, indices_z):
        if np.sqrt(bins_y[y]**2 + bins_z[z]**2)<=300:
            distance_yz.append(np.sqrt(bins_y[y]**2 + bins_z[z]**2))
            distance_xyz.append(np.sqrt(bins_x[x]**2 + bins_y[y]**2 + bins_z[z]**2))
            median.append(histogram[x,y,z])
    distance_x = np.asarray(distance_x)
    distance_yz = np.asarray(distance_yz)
    distance_xyz = np.asarray(distance_xyz)
    median = np.asarray(median)
    distance_x = np.asarray(distance_x)
    return distance_yz, median, distance_x, distance_xyz

In [14]:
bt_left_dist, bt_left_med, bt_left_dx, bt_left_dxyz = distance_and_metric(bitumen_left_hist, np.where(bitumen_left_hist>0), bins_left_bitumen)
bt_right_dist, bt_right_med, bt_right_dx, bt_right_dxyz = distance_and_metric(bitumen_right_hist, np.where(bitumen_right_hist>0), bins_right_bitumen)
air_dist, air_med, air_dx, air_dxyz = distance_and_metric(bubble_hist,np.where(bubble_hist>0), bins_bubble)

In [15]:
all_data_d, all_data_med, all_data_dx, all_data_dxyz = distance_and_metric(histogram_21l, np.where(histogram_21l>0), (bincentre_x, bincentre_y, bincentre_z))

In [16]:
all_data_drum_d, all_data_drum_med, all_data_drum_dx, all_data_drum_dxyz = distance_and_metric(drum_hist, np.where(drum_hist>=0),
                                                                                               bins_drum)

In [17]:
plt.figure(figsize=(10,6))
plt.scatter(all_data_drum_d, all_data_drum_med, c='cornflowerblue', s=5, marker='s', alpha=.5, label='alldata drum (metric>0)')
plt.scatter(bt_left_dist, bt_left_med, c='darkgreen', s=5, marker='s', alpha=.3, label='bitumen left: (x in (-440;-200); y,z in (-300;300))')
#plt.scatter(bt_right_dist, bt_right_med, c='goldenrod', s=10, marker='s', alpha=.3, label='bitumen: (x in (200;440); y,z in (-300;300))')

plt.scatter(air_dist, air_med, c='crimson', s=5, marker='s', alpha=.5, label='bubble: x, y, z in (-100;100)')
plt.axhline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-16,11.25,'11.265', fontsize=8)
plt.ylim(9.5,13)
#plt.ylim(0,15)
#plt.yticks(np.arange(0,15,1))
plt.xlabel('distance from x axis (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
plt.figure(figsize=(10,6))
plt.scatter(all_data_drum_d, all_data_drum_med, c='cornflowerblue', s=10, marker='s', alpha=.5, label='alldata drum (metric>0)')
#plt.scatter(bt_left_dist, bt_left_med, c='darkgreen', s=10, marker='s', alpha=.3, label='bitumen: (x in (-440;-200); y,z in (-300;300))')
plt.scatter(bt_right_dist, bt_right_med, c='goldenrod', s=10, marker='s', alpha=.3, label='bitumen right: (x in (200;440); y,z in (-300;300))')

plt.scatter(air_dist, air_med, c='crimson', s=10, marker='s', alpha=.5, label='bubble: x, y, z in (-100;100)')
plt.axhline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-16,11.25,'11.265', fontsize=8)
plt.ylim(9.5,13)
#plt.xlim(0,1200)
plt.xlabel('distance from x axis (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
plt.figure(figsize=(10,6))
plt.scatter(all_data_drum_dxyz, all_data_drum_med, c='cornflowerblue', s=10, marker='s', alpha=.5, label='alldata drum (metric>0)')
#plt.scatter(bt_left_dist, bt_left_med, c='darkgreen', s=10, marker='s', alpha=.3, label='bitumen: (x in (-440;-200); y,z in (-300;300))')
plt.scatter(bt_right_dxyz, bt_right_med, c='goldenrod', s=10, marker='s', alpha=.3, label='bitumen right: (x in (200;440); y,z in (-300;300))')

plt.scatter(air_dxyz, air_med, c='crimson', s=10, marker='s', alpha=.5, label='bubble: x, y, z in (-100;100)')
plt.axhline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-35,11.25,'11.265', fontsize=8)
plt.ylim(9.5,13)
#plt.xlim(0,1200)
plt.xlabel('distance from (0;0;0) (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
plt.figure(figsize=(10,6))
plt.scatter(all_data_drum_dxyz, all_data_drum_med, c='cornflowerblue', s=10, marker='s', alpha=.5, label='alldata drum (metric>0)')
plt.scatter(bt_left_dxyz, bt_left_med, c='darkgreen', s=10, marker='s', alpha=.3, label='bitumen: (x in (-440;-200); y,z in (-300;300))')
#plt.scatter(bt_right_dxyz, bt_right_med, c='goldenrod', s=10, marker='s', alpha=.3, label='bitumen right: (x in (200;440); y,z in (-300;300))')

plt.scatter(air_dxyz, air_med, c='crimson', s=10, marker='s', alpha=.5, label='bubble: x, y, z in (-100;100)')
plt.axhline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-35,11.25,'11.265', fontsize=8)
plt.ylim(9.5,13)
#plt.xlim(0,1200)
plt.xlabel('distance from (0;0;0) (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
plt.figure(figsize=(10,6))
plt.scatter(all_data_drum_dxyz, all_data_drum_med, c='cornflowerblue', s=10, marker='s', alpha=.5, label='alldata drum (metric>0)')
plt.scatter(bt_left_dxyz, bt_left_med, c='darkgreen', s=10, marker='s', alpha=.3, label='bitumen: (x in (-440;-200); y,z in (-300;300))')
plt.scatter(bt_right_dxyz, bt_right_med, c='goldenrod', s=10, marker='s', alpha=.3, label='bitumen right: (x in (200;440); y,z in (-300;300))')

plt.scatter(air_dxyz, air_med, c='crimson', s=10, marker='s', alpha=.5, label='bubble: x, y, z in (-100;100)')
plt.axhline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-35,11.25,'11.265', fontsize=8)
plt.ylim(9.5,13)
#plt.xlim(0,1200)
plt.xlabel('distance from (0;0;0) (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
bins = np.linspace(10,15, 100)
plt.figure()
plt.hist(drum_hist.ravel(), bins=bins, label='all data - drum')
plt.hist(bitumen_left_hist.ravel(), alpha=.6,bins=bins, label='"bitumen" left')
plt.hist(bitumen_right_hist.ravel(),alpha=.4, bins=bins, label='"bitumen" right')
plt.ylim(0,1200)
plt.hist(bubble_hist.ravel(), bins=bins, label='"bubble"')
plt.legend(loc='upper right')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Bitumen Only

In [23]:
infile_bitumen = 'data/largedrum_onlybitumen_dense_newmetrics_3cmVoxel_April2021.discriminator.root'

In [24]:
fu = uproot3.open(infile_bitumen)
histogram_bitumen = fu['histMedianMetric'].numpy()[0]
bins_x, bins_y, bins_z = fu['histMedianMetric;1'].numpy()[1][0]
shape_x, shape_y, shape_z = histogram_bitumen.shape

In [25]:
bitumen_left_hist, bins_left_bitumen = select_histogram_coords(histogram_bitumen, -440, -200, -300, 300, -300, 300)

In [26]:
bitumen_right_hist, bins_right_bitumen = select_histogram_coords(histogram_bitumen, 200, 440, -300, 300, -300, 300)

In [27]:
bubble_hist, bins_bubble = select_histogram_coords(histogram_bitumen, -100, 100, -100, 100, -100, 100)

In [28]:
drum_hist, bins_drum = select_histogram_coords(histogram_bitumen, -440, 440, -300, 300, -300, 300)

In [29]:
bt_left_dist, bt_left_med, bt_left_dx, bt_left_dxyz = distance_and_metric(bitumen_left_hist, np.where(bitumen_left_hist>0), bins_left_bitumen)
bt_right_dist, bt_right_med, bt_right_dx, bt_right_dxyz = distance_and_metric(bitumen_right_hist, np.where(bitumen_right_hist>0), bins_right_bitumen)
air_dist, air_med, air_dx, air_dxyz = distance_and_metric(bubble_hist,np.where(bubble_hist>0), bins_bubble)

In [30]:
all_data_d, all_data_med, all_data_dx, all_data_dxyz = distance_and_metric(histogram_bitumen, np.where(histogram_bitumen>0), (bincentre_x, bincentre_y, bincentre_z))

In [31]:
all_data_drum_d, all_data_drum_med, all_data_drum_dx, all_data_drum_dxyz = distance_and_metric(drum_hist, np.where(drum_hist>0),
                                                                                               bins_drum)

In [32]:
plt.figure(figsize=(10,6))
plt.scatter(all_data_drum_d, all_data_drum_med, c='cornflowerblue', s=5, marker='s', alpha=.5, label='alldata drum (metric>0)')
plt.scatter(bt_left_dist, bt_left_med, c='darkgreen', s=5, marker='s', alpha=.3, label='bitumen left: (x in (-440;-200); y,z in (-300;300))')
#plt.scatter(bt_right_dist, bt_right_med, c='goldenrod', s=10, marker='s', alpha=.3, label='bitumen: (x in (200;440); y,z in (-300;300))')

plt.scatter(air_dist, air_med, c='crimson', s=5, marker='s', alpha=.5, label='bubble: x, y, z in (-100;100)')
plt.axhline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-16,11.25,'11.265', fontsize=8)
plt.ylim(9.5,13)
#plt.xlim(0,1200)
plt.xlabel('distance from x axis (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
plt.figure(figsize=(10,6))
plt.scatter(all_data_drum_d, all_data_drum_med, c='cornflowerblue', s=10, marker='s', alpha=.5, label='alldata drum (metric>0)')
#plt.scatter(bt_left_dist, bt_left_med, c='darkgreen', s=10, marker='s', alpha=.3, label='bitumen: (x in (-440;-200); y,z in (-300;300))')
plt.scatter(bt_right_dist, bt_right_med, c='goldenrod', s=10, marker='s', alpha=.3, label='bitumen right: (x in (200;440); y,z in (-300;300))')

plt.scatter(air_dist, air_med, c='crimson', s=10, marker='s', alpha=.5, label='bubble: x, y, z in (-100;100)')
plt.axhline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-16,11.25,'11.265', fontsize=8)
plt.ylim(9.5,13)
#plt.xlim(0,1200)
plt.xlabel('distance from x axis (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
plt.figure(figsize=(10,6))
plt.scatter(all_data_drum_dxyz, all_data_drum_med, c='cornflowerblue', s=10, marker='s', alpha=.5, label='alldata drum (metric>0)')
plt.scatter(bt_left_dxyz, bt_left_med, c='darkgreen', s=10, marker='s', alpha=.3, label='bitumen: (x in (-440;-200); y,z in (-300;300))')
#plt.scatter(bt_right_dxyz, bt_right_med, c='goldenrod', s=10, marker='s', alpha=.3, label='bitumen right: (x in (200;440); y,z in (-300;300))')

plt.scatter(air_dxyz, air_med, c='crimson', s=10, marker='s', alpha=.5, label='bubble: x, y, z in (-100;100)')
plt.axhline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-35,11.25,'11.265', fontsize=8)
plt.ylim(9.5,13)
#plt.xlim(0,1200)
plt.xlabel('distance from (0;0;0) (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
plt.figure(figsize=(10,6))
plt.scatter(all_data_drum_dxyz, all_data_drum_med, c='cornflowerblue', s=10, marker='s', alpha=.5, label='alldata drum (metric>0)')
#plt.scatter(bt_left_dxyz, bt_left_med, c='darkgreen', s=10, marker='s', alpha=.3, label='bitumen: (x in (-440;-200); y,z in (-300;300))')
plt.scatter(bt_right_dxyz, bt_right_med, c='goldenrod', s=10, marker='s', alpha=.3, label='bitumen right: (x in (200;440); y,z in (-300;300))')

plt.scatter(air_dxyz, air_med, c='crimson', s=10, marker='s', alpha=.5, label='bubble: x, y, z in (-100;100)')
plt.axhline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-35,11.25,'11.265', fontsize=8)
plt.ylim(9.5,13)
#plt.xlim(0,1200)
plt.xlabel('distance from (0;0;0) (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
plt.figure(figsize=(10,6))
plt.scatter(all_data_drum_dxyz, all_data_drum_med, c='cornflowerblue', s=10, marker='s', alpha=.5, label='alldata drum (metric>0)')
plt.scatter(bt_left_dxyz, bt_left_med, c='darkgreen', s=10, marker='s', alpha=.3, label='bitumen: (x in (-440;-200); y,z in (-300;300))')
plt.scatter(bt_right_dxyz, bt_right_med, c='goldenrod', s=10, marker='s', alpha=.3, label='bitumen right: (x in (200;440); y,z in (-300;300))')

plt.scatter(air_dxyz, air_med, c='crimson', s=10, marker='s', alpha=.5, label='bubble: x, y, z in (-100;100)')
plt.axhline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-35,11.25,'11.265', fontsize=8)
plt.ylim(9.5,13)
#plt.xlim(0,1200)
plt.xlabel('distance from (0;0;0) (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
bins = np.linspace(10,15, 100)
plt.figure()
plt.ylim(0,1200)
plt.hist(drum_hist[drum_hist>0].ravel(), bins=bins, label='all data - drum')
plt.hist(bitumen_left_hist.ravel(), alpha=.7,bins=bins, label='"bitumen" left')
plt.hist(bitumen_right_hist.ravel(),alpha=.7, bins=bins, label='"bitumen" right')
plt.hist(bubble_hist.ravel(), bins=bins, label='"bubble"')
plt.legend(loc='upper right')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Hydrogen only

In [38]:
infile_hydrogen = 'data/largedrum_onlyhydrogen_dense_newmetrics_3cmVoxel_April2021.discriminator.root'

In [39]:
fu = uproot3.open(infile_hydrogen)
histogram_hydrogen = fu['histMedianMetric'].numpy()[0]
bins_x, bins_y, bins_z = fu['histMedianMetric;1'].numpy()[1][0]
shape_x, shape_y, shape_z = histogram_hydrogen.shape

In [40]:
bitumen_left_hist, bins_left_bitumen = select_histogram_coords(histogram_hydrogen, -440, -200, -300, 300, -300, 300)

In [41]:
bitumen_right_hist, bins_right_bitumen = select_histogram_coords(histogram_hydrogen, 200, 440, -300, 300, -300, 300)

In [42]:
bubble_hist, bins_bubble = select_histogram_coords(histogram_hydrogen, -100, 100, -100, 100, -100, 100)

In [43]:
drum_hist, bins_drum = select_histogram_coords(histogram_hydrogen, -440, 440, -300, 300, -300, 300)

In [44]:
bt_left_dist, bt_left_med, bt_left_dx, bt_left_dxyz = distance_and_metric(bitumen_left_hist, np.where(bitumen_left_hist>0), bins_left_bitumen)
bt_right_dist, bt_right_med, bt_right_dx, bt_right_dxyz = distance_and_metric(bitumen_right_hist, np.where(bitumen_right_hist>0), bins_right_bitumen)
air_dist, air_med, air_dx, air_dxyz = distance_and_metric(bubble_hist,np.where(bubble_hist>0), bins_bubble)

In [45]:
all_data_d, all_data_med, all_data_dx, all_data_dxyz = distance_and_metric(histogram_bitumen, np.where(histogram_hydrogen>0), (bincentre_x, bincentre_y, bincentre_z))

In [46]:
all_data_drum_d, all_data_drum_med, all_data_drum_dx, all_data_drum_dxyz = distance_and_metric(drum_hist, np.where(drum_hist>0),
                                                                                               bins_drum)

In [47]:
plt.figure(figsize=(10,6))
plt.scatter(all_data_drum_d, all_data_drum_med, c='cornflowerblue', s=5, marker='s', alpha=.5, label='alldata drum (metric>0)')
plt.scatter(bt_left_dist, bt_left_med, c='darkgreen', s=5, marker='s', alpha=.3, label='bitumen left: (x in (-440;-200); y,z in (-300;300))')
#plt.scatter(bt_right_dist, bt_right_med, c='goldenrod', s=10, marker='s', alpha=.3, label='bitumen: (x in (200;440); y,z in (-300;300))')

plt.scatter(air_dist, air_med, c='crimson', s=5, marker='s', alpha=.5, label='bubble: x, y, z in (-100;100)')
plt.axhline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-16,11.25,'11.265', fontsize=8)
plt.ylim(9.5,13)
#plt.xlim(0,1200)
plt.xlabel('distance from x axis (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
plt.figure(figsize=(10,6))
plt.scatter(all_data_drum_d, all_data_drum_med, c='cornflowerblue', s=10, marker='s', alpha=.5, label='alldata drum (metric>0)')
#plt.scatter(bt_left_dist, bt_left_med, c='darkgreen', s=10, marker='s', alpha=.3, label='bitumen: (x in (-440;-200); y,z in (-300;300))')
plt.scatter(bt_right_dist, bt_right_med, c='goldenrod', s=10, marker='s', alpha=.3, label='bitumen right: (x in (200;440); y,z in (-300;300))')

plt.scatter(air_dist, air_med, c='crimson', s=10, marker='s', alpha=.5, label='bubble: x, y, z in (-100;100)')
plt.axhline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-16,11.25,'11.265', fontsize=8)
plt.ylim(9.5,13)
#plt.xlim(0,1200)
plt.xlabel('distance from x axis (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
plt.figure(figsize=(10,6))
plt.scatter(all_data_drum_dxyz, all_data_drum_med, c='cornflowerblue', s=10, marker='s', alpha=.5, label='alldata drum (metric>0)')
plt.scatter(bt_left_dxyz, bt_left_med, c='darkgreen', s=10, marker='s', alpha=.3, label='bitumen: (x in (-440;-200); y,z in (-300;300))')
#plt.scatter(bt_right_dxyz, bt_right_med, c='goldenrod', s=10, marker='s', alpha=.3, label='bitumen right: (x in (200;440); y,z in (-300;300))')

plt.scatter(air_dxyz, air_med, c='crimson', s=10, marker='s', alpha=.5, label='bubble: x, y, z in (-100;100)')
plt.axhline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-35,11.25,'11.265', fontsize=8)
plt.ylim(9.5,13)
#plt.xlim(0,1200)
plt.xlabel('distance from (0;0;0) (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
plt.figure(figsize=(10,6))
plt.scatter(all_data_drum_dxyz, all_data_drum_med, c='cornflowerblue', s=10, marker='s', alpha=.5, label='alldata drum (metric>0)')
#plt.scatter(bt_left_dxyz, bt_left_med, c='darkgreen', s=10, marker='s', alpha=.3, label='bitumen: (x in (-440;-200); y,z in (-300;300))')
plt.scatter(bt_right_dxyz, bt_right_med, c='goldenrod', s=10, marker='s', alpha=.3, label='bitumen right: (x in (200;440); y,z in (-300;300))')

plt.scatter(air_dxyz, air_med, c='crimson', s=10, marker='s', alpha=.5, label='bubble: x, y, z in (-100;100)')
plt.axhline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-35,11.25,'11.265', fontsize=8)
plt.ylim(9.5,13)
#plt.xlim(0,1200)
plt.xlabel('distance from (0;0;0) (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
plt.figure(figsize=(10,6))
plt.scatter(all_data_drum_dxyz, all_data_drum_med, c='cornflowerblue', s=10, marker='s', alpha=.5, label='alldata drum (metric>0)')
plt.scatter(bt_left_dxyz, bt_left_med, c='darkgreen', s=10, marker='s', alpha=.3, label='bitumen: (x in (-440;-200); y,z in (-300;300))')
plt.scatter(bt_right_dxyz, bt_right_med, c='goldenrod', s=10, marker='s', alpha=.3, label='bitumen right: (x in (200;440); y,z in (-300;300))')

plt.scatter(air_dxyz, air_med, c='crimson', s=10, marker='s', alpha=.5, label='bubble: x, y, z in (-100;100)')
plt.axhline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-35,11.25,'11.265', fontsize=8)
plt.ylim(9.5,13)
#plt.xlim(0,1200)
plt.xlabel('distance from (0;0;0) (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [52]:
bins = np.linspace(10,15, 100)
plt.figure()
plt.ylim(0,1200)
plt.hist(drum_hist[drum_hist>0].ravel(), bins=bins, label='all data - drum')
plt.hist(bitumen_left_hist.ravel(), alpha=.7,bins=bins, label='"bitumen" left')
plt.hist(bitumen_right_hist.ravel(),alpha=.7, bins=bins, label='"bitumen" right')
plt.hist(bubble_hist.ravel(), bins=bins, label='"bubble"')
plt.legend(loc='upper right')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Shifted Bubble

In [68]:
infile_shifted = 'data/largedrum_21L_5cm_dense_newmetrics_3cmVoxel_April2021.discriminator.root'

In [69]:
fu = uproot3.open(infile_bitumen)
histogram_shifted = fu['histMedianMetric'].numpy()[0]
bins_x, bins_y, bins_z = fu['histMedianMetric;1'].numpy()[1][0]
shape_x, shape_y, shape_z = histogram_shifted.shape

In [70]:
bitumen_left_hist, bins_left_bitumen = select_histogram_coords(histogram_shifted, -440, -200, -300, 300, -300, 300)

In [71]:
bitumen_right_hist, bins_right_bitumen = select_histogram_coords(histogram_shifted, 200, 440, -300, 300, -300, 300)

In [72]:
bubble_hist, bins_bubble = select_histogram_coords(histogram_shifted, -100, 100, -100, 100, -100, 100)

In [73]:
drum_hist, bins_drum = select_histogram_coords(histogram_shifted, -440, 440, -300, 300, -300, 300)

In [74]:
bt_left_dist, bt_left_med, bt_left_dx, bt_left_dxyz = distance_and_metric(bitumen_left_hist, np.where(bitumen_left_hist>0), bins_left_bitumen)
bt_right_dist, bt_right_med, bt_right_dx, bt_right_dxyz = distance_and_metric(bitumen_right_hist, np.where(bitumen_right_hist>0), bins_right_bitumen)
air_dist, air_med, air_dx, air_dxyz = distance_and_metric(bubble_hist,np.where(bubble_hist>0), bins_bubble)

In [75]:
all_data_d, all_data_med, all_data_dx, all_data_dxyz = distance_and_metric(histogram_bitumen, np.where(histogram_bitumen>0), (bincentre_x, bincentre_y, bincentre_z))

In [76]:
all_data_drum_d, all_data_drum_med, all_data_drum_dx, all_data_drum_dxyz = distance_and_metric(drum_hist, np.where(drum_hist>0),
                                                                                               bins_drum)

In [77]:
plt.figure(figsize=(10,6))
plt.scatter(all_data_drum_d, all_data_drum_med, c='cornflowerblue', s=5, marker='s', alpha=.5, label='alldata drum (metric>0)')
plt.scatter(bt_left_dist, bt_left_med, c='darkgreen', s=5, marker='s', alpha=.3, label='bitumen left: (x in (-440;-200); y,z in (-300;300))')
#plt.scatter(bt_right_dist, bt_right_med, c='goldenrod', s=10, marker='s', alpha=.3, label='bitumen: (x in (200;440); y,z in (-300;300))')

plt.scatter(air_dist, air_med, c='crimson', s=5, marker='s', alpha=.5, label='bubble: x, y, z in (-100;100)')
plt.axhline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-16,11.25,'11.265', fontsize=8)
plt.ylim(9.5,13)
#plt.xlim(0,1200)
plt.xlabel('distance from x axis (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
plt.figure(figsize=(10,6))
plt.scatter(all_data_drum_d, all_data_drum_med, c='cornflowerblue', s=10, marker='s', alpha=.5, label='alldata drum (metric>0)')
#plt.scatter(bt_left_dist, bt_left_med, c='darkgreen', s=10, marker='s', alpha=.3, label='bitumen: (x in (-440;-200); y,z in (-300;300))')
plt.scatter(bt_right_dist, bt_right_med, c='goldenrod', s=10, marker='s', alpha=.3, label='bitumen right: (x in (200;440); y,z in (-300;300))')

plt.scatter(air_dist, air_med, c='crimson', s=10, marker='s', alpha=.5, label='bubble: x, y, z in (-100;100)')
plt.axhline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-16,11.25,'11.265', fontsize=8)
plt.ylim(9.5,13)
#plt.xlim(0,1200)
plt.xlabel('distance from x axis (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:
plt.figure(figsize=(10,6))
plt.scatter(all_data_drum_dxyz, all_data_drum_med, c='cornflowerblue', s=10, marker='s', alpha=.5, label='alldata drum (metric>0)')
plt.scatter(bt_left_dxyz, bt_left_med, c='darkgreen', s=10, marker='s', alpha=.3, label='bitumen: (x in (-440;-200); y,z in (-300;300))')
#plt.scatter(bt_right_dxyz, bt_right_med, c='goldenrod', s=10, marker='s', alpha=.3, label='bitumen right: (x in (200;440); y,z in (-300;300))')

plt.scatter(air_dxyz, air_med, c='crimson', s=10, marker='s', alpha=.5, label='bubble: x, y, z in (-100;100)')
plt.axhline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-35,11.25,'11.265', fontsize=8)
plt.ylim(9.5,13)
#plt.xlim(0,1200)
plt.xlabel('distance from (0;0;0) (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [80]:
plt.figure(figsize=(10,6))
plt.scatter(all_data_drum_dxyz, all_data_drum_med, c='cornflowerblue', s=10, marker='s', alpha=.5, label='alldata drum (metric>0)')
#plt.scatter(bt_left_dxyz, bt_left_med, c='darkgreen', s=10, marker='s', alpha=.3, label='bitumen: (x in (-440;-200); y,z in (-300;300))')
plt.scatter(bt_right_dxyz, bt_right_med, c='goldenrod', s=10, marker='s', alpha=.3, label='bitumen right: (x in (200;440); y,z in (-300;300))')

plt.scatter(air_dxyz, air_med, c='crimson', s=10, marker='s', alpha=.5, label='bubble: x, y, z in (-100;100)')
plt.axhline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-35,11.25,'11.265', fontsize=8)
plt.ylim(9.5,13)
#plt.xlim(0,1200)
plt.xlabel('distance from (0;0;0) (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
plt.figure(figsize=(10,6))
plt.scatter(all_data_drum_dxyz, all_data_drum_med, c='cornflowerblue', s=10, marker='s', alpha=.5, label='alldata drum (metric>0)')
plt.scatter(bt_left_dxyz, bt_left_med, c='darkgreen', s=10, marker='s', alpha=.3, label='bitumen: (x in (-440;-200); y,z in (-300;300))')
plt.scatter(bt_right_dxyz, bt_right_med, c='goldenrod', s=10, marker='s', alpha=.3, label='bitumen right: (x in (200;440); y,z in (-300;300))')

plt.scatter(air_dxyz, air_med, c='crimson', s=10, marker='s', alpha=.5, label='bubble: x, y, z in (-100;100)')
plt.axhline(11.265, linestyle='--', c='gray', linewidth=1)
plt.text(-35,11.25,'11.265', fontsize=8)
plt.ylim(9.5,13)
#plt.xlim(0,1200)
plt.xlabel('distance from (0;0;0) (mm)')
plt.ylabel('metric')
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [67]:
bins = np.linspace(10,15, 100)
plt.figure()
plt.ylim(0,1200)
plt.hist(drum_hist[drum_hist>0].ravel(), bins=bins, label='all data - drum')
plt.hist(bitumen_left_hist.ravel(), alpha=.7,bins=bins, label='"bitumen" left')
plt.hist(bitumen_right_hist.ravel(),alpha=.7, bins=bins, label='"bitumen" right')
plt.hist(bubble_hist.ravel(), bins=bins, label='"bubble"')
plt.legend(loc='upper right')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …